#Project 2: Content-Based and Collaborative Filtering
By Latif Masud

##Overview
For this project, I broke it into two parts. The first part performs a Content-Based filering recommender system using data from OMDb's API, which then performed a tf-idf filtering. The data used was plots for all seven Harry Potter movies. The second part utlized a User-User Collaborative filtering recommender system overa  randomly generated set of user movie ratings. All the code work was done in Python. 

##Content-Based Filtering
For this part, OMDb API data was put through into `textBlob` and then calculations were performed.

###Libraries
The following libraries were used. `requests` was used to make API calls, wile `math` was used to perform tf-idf calcualations. `unicodedata` was used to convert unicode data that was returned by the API to a Python string. `textblob` was used to make it easy to figure out how much of each word was in a movie plot. 

In [10]:
import requests
import math
import unicodedata
from textblob import TextBlob as tb

###Functions
The `tf` function returns the frequency of a word in a blob. `n_containing` returns the number of movie plots that contain the word. `idf` stands for Inverse Document Frequency, which is the calculation of how frequnently a word appears in a file, which is then inverted. `tfidf` is simply the product of `tf` and `idf`. Lastly, a `clean_words` function is defined to take out any very commonly occuring words.

In [11]:
def tf(word, blob):
    return float(blob.words.count(word)) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.words)

def idf(word, bloblist):
    return float(math.log(float(len(bloblist)) / (1 + n_containing(word, bloblist))))

def tfidf(word, blob, bloblist):
    return float(tf(word, blob)) * float(idf(word, bloblist))

def clean_words (rm_words, query):
    wrds = query.lower()
    for word in rm_words:
        wrds = wrds.replace(word, "")
        
    return wrds

###Code
A few common words such as "harry" and "potter" are taken out of the plots. This is done because the main character names would overwhelm the overall results. 

In [12]:
remove_words = ["harry", "potter", "ron", "weasley", "hermione", "granger", "hogwarts", ","]

An array for all the plot summaries is created, and we also define an array of all the years that the movies came out. 

In [21]:
potter_summaries = []
potter_years = [2011, 2010, 2009, 2007, 2005, 2004,2002, 2001]
potter_titles = [
    "Harry Potter and the Deathly Hallows – Part 2",
    "Harry Potter and the Deathly Hallows – Part 1",
    "Harry Potter and the Half-Blood Prince",
    "Harry Potter and the Order of the Phoenix",
    "Harry Potter and the Goblet of Fire",
    "Harry Potter and the Prisoner of Azkaban",
    "Harry Potter and the Chamber of Secrets",
    "Harry Potter and the Sorcerer's Stone"
]

For each of the movie years, I make an API call to OMDb and get the result back. I then take the plot, convert it into a Python string from unicode. After that, all the words defined in `remove_words` are taken out, and appeneded into the summaries array as `TextBlob`s

In [22]:
for year in potter_years:
    r = requests.get('http://www.omdbapi.com/?apikey=ad492b8&t=harry+potter&y='+str(year)+'&plot=full')
    plot = unicodedata.normalize('NFKD',  r.json()["Plot"]).encode('ascii','ignore')
    potter_summaries.append(tb(clean_words(remove_words, plot)))
    

Once we have all the plots, I run through a loop, and calculate the tf-idf scores for each word. For each file, I get back the top three words. The top three words are found by first sorting the `scores` array, and then reversing it. A for loop is then run for the first three spots of that sorted array. 

In [25]:
for i, blob in enumerate(potter_summaries):
    print("Top words in " + potter_titles[i])
    scores = {word: tfidf(word, blob, potter_summaries) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:3]:
        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))
    print ""

Top words in Harry Potter and the Deathly Hallows – Part 2
	Word: their, TF-IDF: 0.05231
	Word: mission, TF-IDF: 0.02616
	Word: battle, TF-IDF: 0.02616

Top words in Harry Potter and the Deathly Hallows – Part 1
	Word: rest, TF-IDF: 0.04864
	Word: control, TF-IDF: 0.02432
	Word: trio, TF-IDF: 0.02432

Top words in Harry Potter and the Half-Blood Prince
	Word: professor, TF-IDF: 0.04621
	Word: persuades, TF-IDF: 0.0231
	Word: slughorn, TF-IDF: 0.0231

Top words in Harry Potter and the Order of the Phoenix
	Word: ca, TF-IDF: 0.02773
	Word: n't, TF-IDF: 0.01962
	Word: that, TF-IDF: 0.0141

Top words in Harry Potter and the Goblet of Fire
	Word: three, TF-IDF: 0.01874
	Word: magical, TF-IDF: 0.01874
	Word: goblet, TF-IDF: 0.01766

Top words in Harry Potter and the Prisoner of Azkaban
	Word: using, TF-IDF: 0.01912
	Word: will, TF-IDF: 0.01353
	Word: he, TF-IDF: 0.0119

Top words in Harry Potter and the Chamber of Secrets
	Word: terrible, TF-IDF: 0.02888
	Word: dobby, TF-IDF: 0.02888
	Word: 

##User-User Collaborative Filtering
For this portion, a random matrix for five users and ten movies is first generated, and then based on those ratings, recommendations are made based on regression and gradient descent. 

###Libraries
For this section, only two libraries are needed: `numpy` for defining matrices, and performing some of the regression, and `scipy`'s optimize to make complex regressions easier. 

In [42]:
from numpy import *
from scipy import optimize

###Functions

`normalize_ratings` takes in two matrices: the ratings and if the user did rate the movie. Based on this data, the mean is calculated of the ratings, and then normalized. 

In [43]:
def normalize_ratings(ratings, did_rate):
    num_movies = ratings.shape[0]
    
    ratings_mean = zeros(shape = (num_movies, 1))
    ratings_norm = zeros(shape = ratings.shape)
    
    for i in range(num_movies): 
        idx = where(did_rate[i] == 1)[0]
        
        #  Calculate mean 
        ratings_mean[i] = mean(ratings[i, idx])
        ratings_norm[i, idx] = ratings[i, idx] - ratings_mean[i]
    
    return ratings_norm, ratings_mean

`unroll_params` calculates the shapes based on movies and features. It takes in data, the number of users, movies, and features, and then defines a matrix for the number of movies and features. A matrix transposition is then perofrmed. The same is done for the number of features and users. 

In [44]:
def unroll_params(X_and_theta, num_users, num_movies, num_features):
    m = X_and_theta[:num_movies * num_features]
    x = m.reshape((num_features, num_movies)).transpose()
    y = X_and_theta[num_movies * num_features:]
    theta = y.reshape(num_features, num_users ).transpose()
    return x, theta

`calcualte_gradient` calcualates the gradient descent of the variables. We start by unrolling the parms, then calculating the difference of the dot product of x, theta, and the did_rate matrix. This returns the intial predictions, and then we subtract the ratings from it. Finally, the x and theta gradients are produced (did not fully understand the math behind the last two lines). 

In [45]:
def calculate_gradient(X_and_theta, ratings, did_rate, num_users, num_movies, num_features, reg_param):
    X, theta = unroll_params(X_and_theta, num_users, num_movies, num_features)

    # we multiply by did_rate because we only want to consider observations for which a rating was given
    difference = X.dot( theta.T ) * did_rate - ratings
    X_grad = difference.dot( theta ) + reg_param * X
    theta_grad = difference.T.dot( X ) + reg_param * theta

`calculate_cost` returns the total sum of the squared errors of the data that is pushed in. First, the parms are unrolled, then the cost (sum) is calculated, and then it is regularazed, which is getting the error of the sum. 

In [46]:
def calculate_cost(X_and_theta, ratings, did_rate, num_users, num_movies, num_features, reg_param):
    X, theta = unroll_params(X_and_theta, num_users, num_movies, num_features)

    cost = sum( (X.dot( theta.T ) * did_rate - ratings) ** 2 ) / 2
    regularization = (reg_param / 2) * (sum( theta**2 ) + sum(X**2))
    return cost + regularization

##Script

To start, we define the number of users and movies we want to use for our prediction:

In [47]:
num_movies = 10
num_users = 5

After that, a `numpy` random array/matrix is defined of the shape of the two parms defined above and a max value of 10: 

In [48]:
ratings = random.randint(11, size= (num_movies, num_users))

For personalized ratings for me, I first define a matrix of all zeroes, and then manually plug in a few values, and append it to our existing ratings:

In [49]:
latif_ratings = zeros((num_movies, 1))

latif_ratings[0] = 8
latif_ratings[4] = 8
latif_ratings[7] = 3

ratings = append(latif_ratings, ratings, axis=1)

The ratings matrix looks like this: 

In [50]:
print ratings

[[  8.  10.   7.   5.   0.   0.]
 [  0.   3.   3.   4.   4.   9.]
 [  0.   5.   6.   4.   7.   1.]
 [  0.   5.   9.   4.   4.   4.]
 [  8.   8.   2.   4.   6.   8.]
 [  0.   5.   4.   4.   0.   9.]
 [  0.   7.   5.   0.   3.   1.]
 [  3.   8.   9.  10.  10.   4.]
 [  0.   6.   7.   7.   1.   5.]
 [  0.   5.   2.   6.   4.   2.]]


A did rate function is defined, which is simply represents boolean values of whether or not a user rated a movie, which is represented by a value greater than 0. 

In [51]:
did_rate = (ratings != 0)*1

We then normalize our ratings, update the number of users (since this changes when I add my personal values in), and set a number of features. These features can be things such as "Comedy", "Action", etc. 

In [52]:
ratings, ratings_mean = normalize_ratings(ratings, did_rate)
num_users = ratings.shape[1]
num_features = 3

A matrix of random movie features and user preferences are then generated, and a row-wise merge (the `r_` part) is performed to create a matrix that has the features and preferences.

In [53]:
movie_features = random.randn( num_movies, num_features )
user_prefs = random.randn( num_users, num_features )
initial_X_and_theta = r_[movie_features.T.flatten(), user_prefs.T.flatten()]

Then the minimized cost is calculated by plugging in variables into the `scipy` optimize function. fmin_cg minimizes using nonlinear gradients, and a maximum iternation (used for accuray) of 100 is set. 

In [54]:
reg_param = 30

minimized_cost_and_optimal_params = optimize.fmin_cg(
    calculate_cost, 
    fprime=calculate_gradient, 
    x0=initial_X_and_theta,
    args=(ratings, did_rate, num_users, num_movies, num_features, reg_param),
    maxiter=100, disp=True, full_output=True)

TypeError: unsupported operand type(s) for *: 'NoneType' and 'NoneType'

After this, we set the cost and optimal features for just my values, and then calculate all the predictions:

In [55]:
cost, optimal_movie_features_and_user_prefs = minimized_cost_and_optimal_params[1], minimized_cost_and_optimal_params[0]

movie_features, user_prefs = unroll_params(optimal_movie_features_and_user_prefs, num_users, num_movies, num_features)
all_predictions = movie_features.dot( user_prefs.T )

NameError: name 'minimized_cost_and_optimal_params' is not defined

Lastly, I pull out my recommendations: 

In [57]:
predictions_for_latif = all_predictions[:, 0:1] + ratings_mean
print predictions_for_latif

NameError: name 'all_predictions' is not defined

*Note*: The last few parts of the script did not work on Jupyter. The script is included in the GitHub repo, and can be run error free. 

The values generated for random ratings generated for one run are below: 
*Ratings*:
[[  8.   5.   3.   7.   5.   8.]
 [  0.   0.  10.   8.   8.   7.]
 [  0.   8.  10.   8.   3.   1.]
 ..., 
 [  3.   3.   4.  10.   9.   2.]
 [  0.   9.   0.   8.   9.   6.]
 [  0.   0.  10.   5.   8.   0.]]
 
 *Predictions for Latif*:
 [[ 6.        ]
 [ 8.25      ]
 [ 6.        ]
 [ 8.75      ]
 [ 7.4       ]
 [ 6.2       ]
 [ 4.6       ]
 [ 5.16666667]
 [ 8.        ]
 [ 7.66666667]]

In [59]:
#Conclusion

This was a tough assignment. The toughest part was often that the math exceeded my current knowledge level. With that said, the second part of the project- the User-User filtering mechanism- was successfully able to recommend movie ratings for me based on randomized data. The first portion was tougher to make predictions using since most of the top words were always adjectives or common words. My initial goal was to predict the sub-vilian of each movie (since the ultimate villain is always Voldemort) based on how often they appear in the plot, but that unfornuately was not possible as the villian's name did not appear very often. 